In [1]:
import requests, datetime, pickle

URL = 'https://open-api.bingx.com'

descargar_data = False

# Obtener la serie de precios para BTC, ETH y XRP 

En temporalidades: 1d y/o 4h, 2h, 1h, 30m, 15m

data_raw = {<br />
    'BTC': <br />
    {'1d': pd.DataFrame(), '4h': pd.DataFrame(), '2h': pd.DataFrame(), '1h': pd.DataFrame(), '30m': pd.DataFrame(), '15m': pd.<br />DataFrame()
    },<br />
    'ETH': <br />
    {'1d': pd.DataFrame(), '4h': pd.DataFrame(), '2h': pd.DataFrame(), '1h': pd.DataFrame(), '30m': pd.DataFrame(), '15m': pd.<br />DataFrame()
    },<br />
    'XRP':<br />
    {'1d': pd.DataFrame(), '4h': pd.DataFrame(), '2h': pd.DataFrame(), '1h': pd.DataFrame(), '30m': pd.DataFrame(), '15m': pd.<br />DataFrame()
    }<br />
}<br />

In [2]:
from backtesting.fechas import past_timestamp, now_timestamp, retrasar_tres_horas
from backtesting.data import response_to_dataframe
from backtesting.ficheros import crear_directorio

In [3]:
if not descargar_data:
    with open('./data/data_raw.pickle', 'rb') as f:
        data_raw = pickle.load(f)
    activos = list(data_raw.keys())
    temporalidades = list(data_raw[activos[0]].keys())

else:
    path = '/openApi/swap/v2/quote/klines'

    data_1d = True
    data_low_temporality = True

    temporalidades = []
    if data_1d:
        temporalidades.append('1d')
    if data_low_temporality:
        temporalidades.append('4h')
        temporalidades.append('2h')
        temporalidades.append('1h')
        temporalidades.append('30m')
        temporalidades.append('15m')
    if not data_1d and not data_low_temporality:
        exit()

    now = datetime.datetime.now()
    start_time = []
    for t in temporalidades:
        if t == '1d':
            start_time.append(int(past_timestamp(365, 'days', now)))
        elif t == '4h':
            start_time.append(int(past_timestamp(45, 'days', now)))
        elif t == '2h':
            start_time.append(int(past_timestamp(20, 'days', now)))
        elif t == '1h':
            start_time.append(int(past_timestamp(20, 'days', now)))
        elif t == '30m':
            start_time.append(int(past_timestamp(6, 'days', now)))
        elif t == '15m':
            start_time.append(int(past_timestamp(3, 'days', now)))

    limit = 555
    activos = ['BTC', 'ETH', 'XRP']
    service = URL + path
    endTime = int(datetime.datetime.timestamp(datetime.datetime.now()) * 1000)
    data_raw = {}
    for activo in activos:
        data_raw[activo] = {}
        symbol = activo + '-USDT'
        for i, interval in enumerate(temporalidades):
            response = requests.get(service, params={'symbol':symbol, 'interval':interval, 'limit':limit, 'startTime':start_time[i], 'endTime':endTime})
            df = response_to_dataframe(response.json())
            if df is None:
                continue
            df['time'] = df['time'].apply(retrasar_tres_horas)
            df = df.set_index('time')
            data_raw[activo][interval] = df

    crear_directorio('data')
    with open('./data/data_raw.pickle', 'wb') as f:
        pickle.dump(data_raw, f, pickle.HIGHEST_PROTOCOL)

# Aplicar Squeeze Momentum Indicator a cada serie temporal y obtener la temporalidad que tenga mayor correlacion para cada activo

SMI_default = {<br />
    'BTC': <br />
    {'1d': pd.DataFrame(), '4h': pd.DataFrame(), '2h': pd.DataFrame(), '1h': pd.DataFrame(), '30m': pd.DataFrame(), '15m': pd.<br />DataFrame()
    },<br />
    'ETH': <br />
    {'1d': pd.DataFrame(), '4h': pd.DataFrame(), '2h': pd.DataFrame(), '1h': pd.DataFrame(), '30m': pd.DataFrame(), '15m': pd.<br />DataFrame()
    },<br />
    'XRP':<br />
    {'1d': pd.DataFrame(), '4h': pd.DataFrame(), '2h': pd.DataFrame(), '1h': pd.DataFrame(), '30m': pd.DataFrame(), '15m': pd.<br />DataFrame()
    }<br />
<br />
}<br />
<br />
SMI_default_corr = {<br />
    'BTC': <br />
    {'1d': 0, '4h': 0, '2h': 0, '1h': 0, '30m': 0, '15m': 0<br />
    },<br />
    'ETH': <br />
    {'1d': 0, '4h': 0, '2h': 0, '1h': 0, '30m': 0, '15m': 0<br />
    },<br />
    'XRP':<br />
    {'1d': 0, '4h': 0, '2h': 0, '1h': 0, '30m': 0, '15m': 0<br />
    }<br />
<br />
}<br />

In [4]:
from backtesting.indicadores import squeeze_momentum_indicator
from scipy.stats import pearsonr

In [5]:
SMI_default = {}
for activo in activos:
    SMI_default[activo] = {}
    for t in temporalidades:
        df = data_raw[activo][t].iloc[:]
        df = squeeze_momentum_indicator(df)
        SMI_default[activo][t] = df.dropna()
        

In [6]:
SMI_default_corr = {}
for activo in activos:
    SMI_default_corr[activo] = {}
    for t in temporalidades:
        df = SMI_default[activo][t].iloc[:]
        df = df[['value', 'close']]
        SMI_default_corr[activo][t], _ = pearsonr(df['value'], df['close'])

print('Correlaciones')
for activo in activos:
    for t in temporalidades:
        print(f'{activo} {t} {round(SMI_default_corr[activo][t], 2)}')
    print()

Correlaciones
BTC 1d 0.36
BTC 4h 0.57
BTC 2h 0.45
BTC 1h 0.35
BTC 30m 0.07
BTC 15m 0.71

ETH 1d 0.41
ETH 4h 0.47
ETH 2h 0.5
ETH 1h 0.5
ETH 30m 0.21
ETH 15m 0.65

XRP 1d 0.41
XRP 4h 0.34
XRP 2h 0.39
XRP 1h 0.27
XRP 30m 0.05
XRP 15m 0.63



In [7]:
temporalidades_filtradas = {}
for activo in activos:
    temporalidades_filtradas[activo] = {}
    for t in temporalidades:
        if SMI_default_corr[activo][t] > 0.5:
            temporalidades_filtradas[activo][t] = SMI_default_corr[activo][t]

# print('Temporalidades filtradas')
# for activo in activos:
#     for t in temporalidades:
#         print(f'{activo} {round(temporalidades_filtradas[activo][t], 2)}')
#     print()

print('Temporalidades filtradas')
for activo in temporalidades_filtradas:
    print(activo)
    for t in temporalidades_filtradas[activo]:
        print(f'correlacion {t}: {round(temporalidades_filtradas[activo][t], 2)}')
    print()

Temporalidades filtradas
BTC
correlacion 4h: 0.57
correlacion 15m: 0.71

ETH
correlacion 2h: 0.5
correlacion 1h: 0.5
correlacion 15m: 0.65

XRP
correlacion 15m: 0.63



### conclusiones 1:

- Si optimizo los parámetros para 1d, como afectan las demás temporalidades?
- Si optimizo los parámetros para una temporalidad con alta correlación, como afectan las demás temporalidades?
- Que diferencia existe entre aplicar la estrategia con parámetros por default y optimizados?

# Aplicar las condiciones de la estrategia a las temporalidades más correlacionadas y visualizar las últimas entradas

In [8]:
from backtesting.indicadores import backtest_entry

In [9]:
entradas_backtest = {}
for activos in temporalidades_filtradas:
    for t in temporalidades_filtradas[activos]:
        entrada_short, entrada_long = backtest_entry(SMI_default[activos][t])
        dict_key = activos + '_' + t
        if len(entrada_short) > 0:
            dict_key_2 = dict_key + '_short'
            entradas_backtest.update({dict_key_2: entrada_short})
        if len(entrada_long) > 0:
            dict_key_2 = dict_key + '_long'
            entradas_backtest.update({dict_key_2: entrada_long})
        

In [12]:
# tomamos las 10 últimas 
entradas_backtest.keys()
entradas_backtest['BTC_4h_short']

,open,close,high,low,volume,upper_BB,lower_BB,tr,upper_KC,lower_KC,value,squeeze_on
time,,,,,,,,,,,,
2023-04-14 05:00:00,30739.0,30803.0,30829.0,30661.0,6051.73,30756.549496,29639.300504,168.0,30679.127462,29716.722538,429.032143,False
2023-04-14 17:00:00,30321.5,30464.5,30510.0,30270.5,4071.54,30796.903253,29665.696747,239.5,30782.901499,29679.698501,213.856786,False


In [24]:
## LO QUE HICE EN ESTA CELDA FUE CORROBORAR QUE SE CUMPLA EL CRITERIOR DE SQUEEZE
## MIENTRAS CORROBORO LAS SEÑALES EN TRADING VIEW HE NOTADO QUE LA CONDICION DE SQUEEZE
## NO SE CUMPLA TAL CUAL DE FORMA GRAFICA, POR LO QUE ESTOY REVISANDO QUE SE CUMPLA LOGICAMENTE
## Y SI SE CUMPLE

# # chequear que en esta fecha
# import pandas as pd
# cuatro_horas_antes_e1 = pd.to_datetime(entradas_backtest['BTC_4h_short'].index[0]) - datetime.timedelta(hours=4)
# # change to str
# cuatro_horas_antes_e1 = str(cuatro_horas_antes_e1)

# SMI_default['BTC']['4h'].loc[cuatro_horas_antes_e1:entradas_backtest['BTC_4h_short'].index[-1]]#:entradas_backtest['BTC_4h_short'].index[0]


,open,close,high,low,volume,upper_BB,lower_BB,tr,upper_KC,lower_KC,value,squeeze_on
time,,,,,,,,,,,,
2023-04-14 01:00:00,30695.5,30739.0,31006.0,30678.0,8745.36,30648.848647,29678.401353,328.0,30695.761342,29631.488658,434.450000,True
2023-04-14 05:00:00,30739.0,30803.0,30829.0,30661.0,6051.73,30756.549496,29639.300504,168.0,30679.127462,29716.722538,429.032143,False
2023-04-14 09:00:00,30803.0,30244.5,30853.5,30098.0,15389.18,30762.123767,29647.226233,755.5,30804.431560,29604.918440,325.420000,True
2023-04-14 13:00:00,30244.5,30321.5,30353.5,29973.5,8435.70,30773.272207,29662.277793,380.0,30812.121276,29623.428724,244.346607,True
2023-04-14 17:00:00,30321.5,30464.5,30510.0,30270.5,4071.54,30796.903253,29665.696747,239.5,30782.901499,29679.698501,213.856786,False


# Optimizar los parámetros del SMI para cada activo en la temporalidad de mayor correlación

SMI_optimized = {<br />
    'BTC': <br />
    {'best_temporality': pd.DataFrame()},<br />
    'ETH': <br />
    {'best_temporality': pd.DataFrame()},<br />
    'XRP':<br />
    {'best_temporality': pd.DataFrame()}<br />
<br />
}<br />

# Aplicar los criterios de la estrategia para cada configuración del indicador y visualizar las señales